In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-25 19:20:18--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.64MB/s    in 0.6s    

2021-03-25 19:20:20 (1.64 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Import SparkSessions
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
#Load the Amazon data into Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"

#Read the Review Dataset as a DataFrame
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

In [9]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to 
#or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on

df_20_orabove = df.filter(df.total_votes >=20)
df_20_orabove.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [13]:
#Filter the new DataFrame or table created in the previous step and create a new DataFrame or table to retrieve all
#the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
df_greater_50 = df_20_orabove.filter((df_20_orabove.helpful_votes/df_20_orabove.total_votes) >= 0.5)
df_greater_50.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [15]:
#Filter the DataFrame or table created in the previous step, and create a new DataFrame or table that retrieves 
#all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_paid_df = df_greater_50.filter(df_greater_50.vine == 'Y')
vine_paid_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [16]:
#Repeat previous step, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_unpaid_df = df_greater_50.filter(df_greater_50.vine == 'N')
vine_unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [28]:
#Determine the total number of paid and unpaid reviews, 
#the number of paid and unpaid 5-star reviews, 
#and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
paid=vine_paid_df.count()
paid

0

In [29]:
unpaid=vine_unpaid_df.count()
unpaid

4532

In [22]:
paid_5 = vine_paid_df.filter(vine_paid_df.star_rating==5).count()
paid_5

0

In [24]:
unpaid_5 = vine_unpaid_df.filter(vine_unpaid_df.star_rating==5).count()
unpaid_5

2507

In [31]:
paid_percent = (paid_5/paid)*100 #gives error because can't divide by zero, but there were zero reviews 
paid_percent

ZeroDivisionError: ignored

In [32]:
unpaid_percent = (unpaid_5/unpaid)*100
unpaid_percent

55.31774051191527